# Análisis exploratorio de datos sobre el comportamiento de personas introvertidas y extrovertidas

Los datos a analizar proceden de Kaggle. Su autor, Rakesh Kapilavayi, indica que los datos fueron recogidos de diferentes encuestas hechas con Google (*Google Forms*) para un proyecto de investigación universitario centrado en los rasgos de personalidad y en las tendencias de comportamiento de los estudiantes.

La base de datos está formada por ocho columnas clasificadas en:
- **Columnas categóricas:**
    - **Stage_fear:** indica si tienen o no pánico escénico.
    - **Drained_after_socializing:** indica si después de socializar se quedan agotados o no. 
    - **Personality:** indica si su personalidad es introvertida o extrovertida. Se trata de la columna objetivo (*target).

- **Columnas numéricas:**
    - **Time_spent_Alone:** tiempo que pasan solos en una escala de 0 a 11.
    - **Social_event_attendance:** asistencia a eventos sociales en una escala de 0 a 10.
    - **Going_outside:** frecuencia de salir fuera en una escala de 0 a 7.
    - **Friends_circle_size:** número de amigos cercanos.
    - **Post_frequency:** frecuencia de postear en redes sociales en una escala de 0 a 10.


    

### EDA - Estructura a seguir:
#### 0. Importaciones necesarias y carga de datos
#### 1. Información básica de los datos
#### 2. Revisión de nulos y duplicados
#### 3. Limpieza y transformación
#### 4. Análisis y visualizaciones

### 0. Importaciones necesarias y carga de datos

In [27]:
# Librerias necesarias:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Carga del csv:
df = pd.read_csv('../data/personality_dataset.csv')

### 1. Información básica de los datos

Para comenzar con en el análisis, es necesario revisar su estructura para saber cuántas observaciones lo componen y que tipo de variables se incluyen. Para ello, se utilizan los métodos shape e info, con los cuales obtenemos información sobre el número de filas y columnas, así como su tipo y la presencia de valores nulos.

De este modo, observamos que el dataset esta formado por 2900 filas y 8 columnas. Entre ellas encontramos tres variables categóricas de tipo *objeto* y 5 variables numéricas de tipo *floats*. Cabe destacar la existencia de nulos en todas las columnas a excepción de *Personality*.

In [29]:
df.shape

(2900, 8)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Time_spent_Alone           2837 non-null   float64
 1   Stage_fear                 2827 non-null   object 
 2   Social_event_attendance    2838 non-null   float64
 3   Going_outside              2834 non-null   float64
 4   Drained_after_socializing  2848 non-null   object 
 5   Friends_circle_size        2823 non-null   float64
 6   Post_frequency             2835 non-null   float64
 7   Personality                2900 non-null   object 
dtypes: float64(5), object(3)
memory usage: 181.4+ KB


A continuación, se muestran las primeras filas del datset para familiarizarnos con los datos que se van a trabajar. También se procede a separar las variables en categóricas y numéricas, así como a identificar la columna objetivo, que permitirá, en análisis futuros, predecir la personalidad de una persona en base al resto de características.

In [31]:
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [32]:
def clasificador_variables(df,target):
    numericas = []
    categoricas = []
    for i in df.columns:
        if i == target:
            target = i    
        elif (df[i].dtype in ['int64', 'float64']) and (i!= target ):
            numericas.append(i)  
        elif (i not in numericas) and (i != target):
            categoricas.append(i)
            
    return target,numericas, categoricas


In [33]:
target, numericas, categoricas = clasificador_variables(df, 'Personality')

In [34]:
print(f'Columna objetivo: {target}')
print(f'Columnas numéricas: {numericas}')
print(f'Columnas categóricas: {categoricas}')

Columna objetivo: Personality
Columnas numéricas: ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
Columnas categóricas: ['Stage_fear', 'Drained_after_socializing']


Es necesario entender cómo se distribuyen las variables numéricas antes de llegar al análisis de comportamientos más complejos. Por lo que se utiliza el método *describe* para que nos proporcione información sobre la tendencia central (media y mediana), dispersión (desviación estándar), valores mínimos y máximos y los cuartiles que dividen la distribución.

De este modo, las principales conclusiones que obtenemos sobre dichas variables son las siguientes:
- **Time_spent_Alone**: su media y mediana presentan valores próximos por lo que no hay sesgo fuerte. Sin embargo, existe mucha variabilidad por lo que el comportamiento de los individuos indica diferencias importantes.
- **Social_event_attendance**: presenta una distribución sesgada ligeramente hacia abajo(mediana menor que la media). También hay mucha variabilidad, lo que indica perfiles distintos en la muestra.
- **Going_outside**: tiene una distribución equilibrada y con menor variabilidad que las anteriores, por lo que el comportamiento hacia salir fuera es más uniforme.
- **Friends_circle_size**: existe sesgo hacia valores superiores. Su alta variabilidad indica diferencias muy marcadas entre personas con círculos de amigos pequeños y grandes.
- **Post_frequency**: la mediana es mayor que la media, por lo que se obtiene un sesgo ligeramente positivo. La frecuencia de publicaciones es heterogénea, existiendo gran variabilidad a la hora de publicar.

Por lo tanto, destaca su gran variabilidad en todas las variables lo cual lo denota tanto su desviación estándar como sus rangos entre mínimos y máximos. En cuanto a su distribución, mayormente son simétricas a excepción de *Friends_circle_size* que es la que mayor sesgo presenta hacia valores altos. Todo esto refleja la heterogeneidad de comportamientos sociales que se analizará con posterioridad.

In [35]:
df.describe()

,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,2837.000000,2838.000000,2834.000000,2823.000000,2835.000000
mean,4.505816,3.963354,3.000000,6.268863,3.564727
std,3.479192,2.903827,2.247327,4.289693,2.926582
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,3.000000,1.000000
50%,4.000000,3.000000,3.000000,5.000000,3.000000
75%,8.000000,6.000000,5.000000,10.000000,6.000000
max,11.000000,10.000000,7.000000,15.000000,10.000000


### 2. Revisión de nulos y duplicados

En este apartado nos centraremos en analizar la cantidad de nulos y duplicados y se decidirá si deben eliminarse, imputarse o dejarse sin modificar. Tratar nulos y duplicados evita la existencia de sesgos, permitiendo que el análisis sea más fiable y facilitando la correcta visualización.

#### 2.1 Valores nulos
Antes de tomar una decisión sobre el tratamiento de los valores nulos, debemos contabilizarlos y obtener su porcentaje por columna. 

A excepción de la columna objetivo que no presenta ningún valor nulo, el resto de variables muestran bajos porcentajes,ninguno superior el 3%. Por lo tanto, será necesario aplicar imputaciones para completar los datos faltantes.

In [36]:
df.isna().sum()

Time_spent_Alone             63
Stage_fear                   73
Social_event_attendance      62
Going_outside                66
Drained_after_socializing    52
Friends_circle_size          77
Post_frequency               65
Personality                   0
dtype: int64

In [37]:
(df.isna().mean()*100).round(2)

Time_spent_Alone             2.17
Stage_fear                   2.52
Social_event_attendance      2.14
Going_outside                2.28
Drained_after_socializing    1.79
Friends_circle_size          2.66
Post_frequency               2.24
Personality                  0.00
dtype: float64

#### 2.1 Valores duplicados
Al igual que en el caso anterior, también tenemos que tener en cuenta si existen filas duplicadas ya que su existencia provoca sesgos porque se pueden estar sobrerrepresentando ciertos valores y por ende, afecta a valores estadísticos como la media, la desviación estándar, etc

En este caso observamos que hay 388 duplicados, lo que supone un 13,38%. sin embargo, cabe destacar que en este dataset no son duplicados realmente sino personas que en la encuesta han registrado los mismo datos que otros. Por lo tanto, eliminarlos implicaría la pérdida de registros válidos.

In [38]:
duplicados = df.duplicated().sum()
porcentaje_duplicados = ((duplicados/len(df))*100).round(2)

print(f'Cantidad de duplicados = {duplicados}')
print(f'Porcentaje de duplicados = {porcentaje_duplicados}')

Cantidad de duplicados = 388
Porcentaje de duplicados = 13.38


In [39]:
df[df.duplicated()]

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
47,10.0,Yes,1.0,2.0,Yes,2.0,0.0,Introvert
217,5.0,Yes,2.0,0.0,Yes,2.0,0.0,Introvert
246,9.0,Yes,0.0,1.0,Yes,2.0,1.0,Introvert
248,9.0,Yes,0.0,2.0,Yes,3.0,2.0,Introvert
254,7.0,Yes,0.0,0.0,Yes,3.0,2.0,Introvert
...,...,...,...,...,...,...,...,...
2884,11.0,Yes,0.0,2.0,Yes,3.0,1.0,Introvert
2890,8.0,Yes,2.0,0.0,Yes,1.0,2.0,Introvert
2891,6.0,Yes,3.0,1.0,Yes,5.0,1.0,Introvert
2892,9.0,Yes,2.0,0.0,Yes,1.0,2.0,Introvert


### 3. Limpieza y transformación

El primer paso de limpieza es normalizar los títulos de las columnas convirtiéndolos todos a minúsculas. Esto evitará errores al referenciar columnas en futuras operaciones.

In [40]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['time_spent_alone', 'stage_fear', 'social_event_attendance',
       'going_outside', 'drained_after_socializing', 'friends_circle_size',
       'post_frequency', 'personality'],
      dtype='object')


Al haber realizado esta mofificacións será necesario ejecutar de nuevo la función definida con anterioridad *clasificador_variables* para obtener los nombres de las columnas modificados y realizar correctamente los siguientes pasos del análisis.

In [42]:
target, numericas, categoricas = clasificador_variables(df, 'personality')

print(f'Columna objetivo: {target}')
print(f'Columnas numéricas: {numericas}')
print(f'Columnas categóricas: {categoricas}')


Columna objetivo: personality
Columnas numéricas: ['time_spent_alone', 'social_event_attendance', 'going_outside', 'friends_circle_size', 'post_frequency']
Columnas categóricas: ['stage_fear', 'drained_after_socializing']


#### 3.1 Variables categóricas

El primer paso a realizar es contar los valores de cada categoría. Esto nos permitirá ver la necesidad de normalizar los datos así como la existencia de valores únicos. Como la variable objetivo también presenta valores de este tipo crearemos una variable nueva para facilitar el análisis.

In [56]:
target_categoricas = [target] + categoricas

print(target_categoricas)

['personality', 'stage_fear', 'drained_after_socializing']


In [57]:
def contar_valores_categoricos(df,columnas):
    for i in columnas:
        print(f'Columna {i}:')
        print(df[i].value_counts(dropna = False))
        print('\n')

In [58]:
contar_valores_categoricos(df,target_categoricas)

Columna personality:
personality
Extrovert    1491
Introvert    1409
Name: count, dtype: int64


Columna stage_fear:
stage_fear
No     1417
Yes    1410
NaN      73
Name: count, dtype: int64


Columna drained_after_socializing:
drained_after_socializing
No     1441
Yes    1407
NaN      52
Name: count, dtype: int64




En cuanto a las transformaciones de estas columnas, para noralizar los datos, también es recomendable pasarlo todo a minúscula y eliminar cualquier posible espacio que haya delante o detrás de los valores. Otro cambio que sería necesario para aplicar modelos predictivos sería pasar todo a 0 y 1. Sin embargo, para que las visualizaciones posteriores se vean con mayor claridad omitiremos este paso.

In [59]:
def normalizar_valores_categoricos (df,columnas):
    for i in columnas:
        df[i] = df[i].str.lower().str.strip()
    return df

In [61]:
df = normalizar_valores_categoricos(df,target_categoricas)

In [62]:
df.head()

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality
0,4.0,no,4.0,6.0,no,13.0,5.0,extrovert
1,9.0,yes,0.0,0.0,yes,0.0,3.0,introvert
2,9.0,yes,1.0,2.0,yes,5.0,2.0,introvert
3,0.0,no,6.0,7.0,no,14.0,8.0,extrovert
4,3.0,no,9.0,4.0,no,8.0,5.0,extrovert


Una vez normalizados todos los datos, procedemos a ver los nulos en detalle para decidir por qué valor vamos a imputarlo. Para optimizar el código usaremos la función definida *contar_nulos*.

In [65]:
def contar_nulos(df,columnas):
    for i in columnas:
        nulos = df[i].isna().sum()
        print(f'Valores nulos de {i} = {nulos}')

In [66]:
contar_nulos(df,categoricas)

Valores nulos de stage_fear = 73
Valores nulos de drained_after_socializing = 52


Para ambas categorías se decide agruparlas según la columna objetivo y obtener así la distribución. A rasgos generales, las personas que tienen pánico escénico suelen ser introvertidas así como aquellas que se sienten agotadas después de socializar.

In [15]:
df.groupby('personality')['stage_fear'].value_counts(dropna = False)

personality  stage_fear
Extrovert    No            1338
             Yes            111
             NaN             42
Introvert    Yes           1299
             No              79
             NaN             31
Name: count, dtype: int64

In [16]:
df.groupby('personality')['drained_after_socializing'].value_counts(dropna = False)

personality  drained_after_socializing
Extrovert    No                           1362
             Yes                           111
             NaN                            18
Introvert    Yes                          1296
             No                             79
             NaN                            34
Name: count, dtype: int64

La siguiente cuestión que se plantea es si todas las personas que tienen pánico escénico se sienten agotadas al socializar y viceversa. Por lo que se agrupan ambas columnas en función de la personalidad.
Los valores obtenidos nos permiten confirmar que todos los estudiantes que se sienten agotados despues de socializar tienen pánico escénico y que todos aquellos que no se sienten agotados al socializar no lo tienen, por lo que podremos imputar valores nulos aplicando esta lógica en ambos sentidos.

In [19]:
df.groupby(['drained_after_socializing','stage_fear'])['personality'].value_counts()

drained_after_socializing  stage_fear  personality
No                         No          Extrovert      1320
                                       Introvert        79
Yes                        Yes         Introvert      1266
                                       Extrovert       111
Name: count, dtype: int64

In [ ]:
df[(df['stage_fear'] == 'Yes')&(df['drained_after_socializing'] == 'No')]

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality


Antes de proceder a las transformaciones, vemos como se distribuyen los nulos dentro de la columna *drained_after_socializing* en función de los valores de la columna *stage_fear*. De este modo, se observa que la mayoría de ellos corresponden a aquellos que presentan pánico escénico.

In [69]:
stage_yes = df[(df['stage_fear'] == 'yes') & (df['drained_after_socializing'].isna())].shape[0]
stage_no = df[(df['stage_fear'] == 'no') & (df['drained_after_socializing'].isna())].shape[0]

print(f'Si "stage_fear" es "yes" hay {stage_yes} nulos de la columna "drained_after_socializing".')
print(f'Si "stage_fear" es "no" hay {stage_no} nulos de la columna "drained_after_socializing".')

Si "stage_fear" es "yes" hay 33 nulos de la columna "drained_after_socializing".
Si "stage_fear" es "no" hay 18 nulos de la columna "drained_after_socializing".


Aplicando la misma lógica, observamos que la mayoría de nulos también se encuentran en aquellos que se sienten agotados después de socializar.

In [71]:
drained_yes = df[(df['drained_after_socializing'] == 'yes') & (df['stage_fear'].isna())].shape[0]
drained_no = df[(df['drained_after_socializing'] == 'no') & (df['stage_fear'].isna())].shape[0]

print(f'Si "drained_after_socializing" es "yes" hay {drained_yes} nulos de la columna "stage_fear".')
print(f'Si "drained_after_socializing" es "no" hay {drained_no} nulos de la columna "stage_fear".')

Si "drained_after_socializing" es "yes" hay 30 nulos de la columna "stage_fear".
Si "drained_after_socializing" es "no" hay 42 nulos de la columna "stage_fear".


Por último, aplicamos los cambios siguiendo la lógica de si se siente agotado tiene pánico escénico y viceversa para ambas columnas y comprobamos si queda algún nulo para ver cómo lo tratamos.

In [76]:
# Transformaciones para stage_fear:
df.loc[(df['drained_after_socializing'] == 'yes') & (df['stage_fear'].isna()),'stage_fear'] = 'yes'
df.loc[(df['drained_after_socializing'] == 'no') & (df['stage_fear'].isna()),'stage_fear'] = 'no'

In [75]:
# Transformaciones para drained_after_socializing:
df.loc[(df['stage_fear'] == 'yes') & (df['drained_after_socializing'].isna()),'drained_after_socializing'] = 'yes'
df.loc[(df['stage_fear'] == 'no') & (df['drained_after_socializing'].isna()),'drained_after_socializing'] = 'no'

In [77]:
contar_nulos(df,categoricas)

Valores nulos de stage_fear = 1
Valores nulos de drained_after_socializing = 1


El resultado obtenido tras las imputaciones es un nulo en cada columna, lo cual se traduce en una sola fila con ambas columnas nulas. 

Como el patrón observado en los datos indica que la mayoría de los casos introvertidos presentan 'yes' en ambas columnas, se imputan estos valores para mantener la coherencia del dataset.

In [35]:
df.loc[df.stage_fear.isna()]

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality
1517,4.0,NaN,3.0,0.0,NaN,2.0,0.0,Introvert


In [82]:
df.loc[df['stage_fear'].isna() & df['drained_after_socializing'].isna(), 
       ['stage_fear', 'drained_after_socializing']] = 'yes'

In [83]:
contar_nulos(df,categoricas)

Valores nulos de stage_fear = 0
Valores nulos de drained_after_socializing = 0


#### 3.1 Variables numéricas

In [41]:
def medias_numericas_target (df,columnas,target):
    for i in columnas:
        print(f'{i} según {target}:')
        print(df.groupby(target)[i].mean().round())
        print('\n')

In [47]:
target = 'personality'
numericas = [i for i in df.columns if (i not in categoricas) and (i != target)]

In [48]:
numericas

['time_spent_alone',
 'social_event_attendance',
 'going_outside',
 'friends_circle_size',
 'post_frequency']

In [49]:
medias_numericas_target(df,numericas,target)

time_spent_alone según personality:
personality
Extrovert    2.0
Introvert    7.0
Name: time_spent_alone, dtype: float64


social_event_attendance según personality:
personality
Extrovert    6.0
Introvert    2.0
Name: social_event_attendance, dtype: float64


going_outside según personality:
personality
Extrovert    5.0
Introvert    1.0
Name: going_outside, dtype: float64


friends_circle_size según personality:
personality
Extrovert    9.0
Introvert    3.0
Name: friends_circle_size, dtype: float64


post_frequency según personality:
personality
Extrovert    6.0
Introvert    1.0
Name: post_frequency, dtype: float64




In [58]:
def moda_numericas_target (df,columnas,target):
    for i in columnas:
        print(f'Moda de {i} según {target}:')
        print(df.groupby(target)[i].agg(lambda x: x.mode()))
        print('\n')

In [59]:
moda_numericas_target(df,numericas,target)

Moda de time_spent_alone según personality:
personality
Extrovert    3.0
Introvert    9.0
Name: time_spent_alone, dtype: float64


Moda de social_event_attendance según personality:
personality
Extrovert    4.0
Introvert    2.0
Name: social_event_attendance, dtype: float64


Moda de going_outside según personality:
personality
Extrovert    5.0
Introvert    0.0
Name: going_outside, dtype: float64


Moda de friends_circle_size según personality:
personality
Extrovert           8.0
Introvert    [2.0, 3.0]
Name: friends_circle_size, dtype: object


Moda de post_frequency según personality:
personality
Extrovert    7.0
Introvert    2.0
Name: post_frequency, dtype: float64




In [72]:
def comparativa_media_mediana_moda(df,columnas,target):
    for i in columnas:
        print(f'Media de{i} según {target}:')
        print(df.groupby(target)[i].mean().round())
        print('\n')

        print(f'Mediana de {i} según {target}:')
        print(df.groupby(target)[i].agg(lambda x: x.median()))
        print('\n')
        
        print(f'Moda de {i} según {target}:')
        print(df.groupby(target)[i].agg(lambda x: x.mode()))
        print('\n---------------------------------------------------------\n')
    

In [74]:
comparativa_media_mediana_moda(df,numericas,target)

Media detime_spent_alone según personality:
personality
Extrovert    2.0
Introvert    7.0
Name: time_spent_alone, dtype: float64


Mediana de time_spent_alone según personality:
personality
Extrovert    2.0
Introvert    7.0
Name: time_spent_alone, dtype: float64


Moda de time_spent_alone según personality:
personality
Extrovert    3.0
Introvert    9.0
Name: time_spent_alone, dtype: float64

---------------------------------------------------------

Media desocial_event_attendance según personality:
personality
Extrovert    6.0
Introvert    2.0
Name: social_event_attendance, dtype: float64


Mediana de social_event_attendance según personality:
personality
Extrovert    6.0
Introvert    2.0
Name: social_event_attendance, dtype: float64


Moda de social_event_attendance según personality:
personality
Extrovert    4.0
Introvert    2.0
Name: social_event_attendance, dtype: float64

---------------------------------------------------------

Media degoing_outside según personality:
personali